https://github.com/Berkmann18/facebook-friend-network
に従うことでfacebook friendsのcsv facebook.csvが生成される。
python codeはjupyterからだとうまく動作しなかったので、vs codeから推奨

In [ ]:
library(ggnetwork)

In [ ]:
library(tidyverse)

In [ ]:
library(readr)

In [ ]:
df = read_tsv("two_column_df.tsv")

In [ ]:
df %>% head

In [ ]:
library(igraph)

In [ ]:
run_igraph = graph.data.frame(df %>% select(Source ,Target))

## nodeに媒介中心性(betweenness centrality)をアサイン

全てのvertex pair間の最短経路のいくつが自身を通るか


In [ ]:
run_bet =  betweenness(run_igraph)

In [ ]:
run_igraph = set_vertex_attr(run_igraph , "bet" , value = run_bet)

## nodeにpage rankをアサイン

In [ ]:
run_page_rank = page_rank(run_igraph)

In [ ]:
run_igraph = set_vertex_attr(run_igraph , "page_rank" , value = run_page_rank$vector)

## nodeにcluster_edge_betweennessをアサイン

https://yokkuns.hatenadiary.org/entry/20120819/1345380060

- 全てのvertex pair間の最短経路のいくつが自身（この場合edge）を通るか、がedge betweenness
- edge betweennessが最大のedgeから取り除いていく
- 全部取り除くところまで終了すると、デンドログラム（樹形図）ができる
- 下から順番にどこかの段階で止めることでコミュニティ分解できる
- どこで止めるかはmodularity Qという尺度がある模様
- igraphのcluster_edge_betweennessではmodularityに基づいてcommunityを決定するのがデフォルト（？）

In [ ]:
run_comm = cluster_edge_betweenness(run_igraph)

In [ ]:
plot_dendrogram(run_comm)

In [ ]:
class(run_comm)

In [ ]:
run_mem = membership(run_comm)

In [ ]:
run_igraph = set_vertex_attr(run_igraph , "comm" , value = run_mem)

### 違う設定をためしてみる（sandbox)

In [ ]:
run_comm_tmp = cluster_edge_betweenness(run_igraph , modularity = F , membership = F)

In [ ]:
class(run_comm_tmp)

In [ ]:
run_comm_tmp

## sigmajs (better than ggplot + ggnetwork + ggiraph)

https://github.com/jacomyal/sigma.js/wiki/Settings

http://sigmajs.john-coene.com/

In [ ]:
library(sigmajs)

In [ ]:
run_igraph

In [ ]:
run_igraph2 = igraph::set_vertex_attr(run_igraph , "label" , value = 
                                      V(run_igraph)$name)


In [ ]:
sigmajs() %>% sigmajs::sg_from_igraph(run_igraph2) %>%   sg_cluster() %>% # cluster
  sg_drag_nodes() %>% # allows user to drag nodes
  sg_neighbours()  %>% 
  sg_settings(drawLabels = FALSE, 
              drawEdgeLabels = FALSE , 
             edgeHoverExtremities = T)

## ggnetwork化

In [ ]:
theme_set(theme_void())

In [ ]:

options(repr.plot.width=14, repr.plot.height=10)

In [ ]:
run_ggdf = fortify(run_igraph , layout = layout_(run_igraph , nicely()))

In [ ]:
run_ggdf = fortify(run_igraph , layout = layout_(run_igraph , with_lgl()))

In [ ]:
(g = ggplot(run_ggdf , aes(x = x , y = y , xend = xend , yend = yend , color = as.character(comm))) + 
 geom_edges(alpha = 0.1 , size = 0.3) +
 #geom_nodes() + 
 guides(color = F))

http://curleylab.psych.columbia.edu/netviz/netviz1.html#/48

In [ ]:
library(ggiraph)

In [ ]:
gr = g + geom_point_interactive(aes(tooltip = name, size = bet))

In [ ]:
run_gi = girafe(ggobj = gr , width_svg = 10, height_svg = 10)

run_gi <- girafe_options(run_gi,
  opts_sizing(rescale = FALSE) )

run_gi

In [ ]:
### 媒介中心性　vs.  page rank

In [ ]:
gr = ggplot(run_ggdf , aes(x = page_rank , y = bet , color = as.character(comm), tooltip = name)) + 
geom_point_interactive() + theme_minimal() + guides(color = F)

In [ ]:
run_gi = girafe(ggobj = gr , width_svg = 10, height_svg = 10)

run_gi <- girafe_options(run_gi,
  opts_sizing(rescale = FALSE) )

run_gi

In [ ]:
run_ggdf %>% head